# Day 11

In [466]:
from copy import deepcopy
from queue import Queue
from collections import defaultdict
from itertools import combinations

def ElevatorMoves(E,nfloors=4):
    '''Returns possibile Elevator movements given Elevator position'''
    return [ E+i for i in [1,-1] if -1<E+i<nfloors and E>-1 and E<nfloors]

def FloorToString(F):
    '''Return a (hashable) string from Floor list to save visited states'''
    s = "|"
    for f in F:
        s += "-".join(sorted(f)) + "|" # sets needs sorting to guarantee uniqueness of visited state
    return(s)

def Fried(f):
    '''Floor configuration checker'''   
    M = { o[0] for o in f if o[1]=="M" }
    G = { o[0] for o in f if o[1]=="G" }
    if len(G) and len(M-G):
        return True
    else:
        return False

def findMoves(F,savepath=False):

    # Define completion condition (all items on last floor)
    ffinal = set()
    for f in F:
        for o in f:
            ffinal.add(o)
    
    # Elevator position
    E = 0 

    # BFS Queue
    queue = Queue()
    if savepath: # saving all steps for debugging purpose
        queue.put([(F,E)])
    else: # Saving only last step floor status, elevator position, elevator moves
        queue.put((F,E,0)) 
    
    # Visited configuration dictionary
    visited = defaultdict(bool) 
    visited[ FloorToString(F)+"E"+str(E) ] = True

    s_longest = 0

    while not queue.empty():

        if savepath:
            path = queue.get()
            F,E = path[-1]
            s = len(path)-1
        else:
            F,E,s = queue.get()
    
        if s>s_longest:
            print(s,end = " ")
            s_longest = s
    
        if F[-1]==ffinal: # all objects collected on 4th floor
            print("")
            return s
        
        # Try to move some objects
        for i in (1,2): # Elevator can bring 1 or 2 objects
            
            for p in combinations(F[E],i):
                
                if Fried(p): # elevator brings frying M+G combination 
                    continue
            
                for e in ElevatorMoves(E): # possible elevator moves
                    
                    Fnew = deepcopy(F)

                    for o in p: # move objects between old and new floor
                        Fnew[e].add(o)
                        Fnew[E].remove(o)
                        
                    if Fried(Fnew[e]) or Fried(Fnew[E]): # new configuration invalid
                        continue          
            
                    if visited[ FloorToString(Fnew)+"E"+str(e) ]: # configuration already checked
                        continue
                    else:
                        if savepath:
                            newpath = deepcopy(path)
                            newpath.append((Fnew,e))
                            queue.put(newpath)
                        else:
                            queue.put((Fnew,e,s+1))
                        visited[ FloorToString(Fnew)+"E"+str(e) ] = True

In [467]:
from timeit import default_timer as timer

F = [ {"HM","LM"}, {"HG"}, {"LG"}, set() ] 

start = timer()
s = findMoves(F)
stop = timer()

print("Test: objects collected on 4th floor in {} moves".format(s))
print("Elapsed time = {:.2f} ms".format((1000.*(stop-start))))

1 2 3 4 5 6 7 8 9 10 11 
Test: objects collected on 4th floor in 11 moves
Elapsed time = 46.55 ms


## Part 1

Parsing the input by hand...

In [468]:
with open("data/input11.txt") as f:
    for l in f.readlines():
        print(l)

The first floor contains a promethium generator and a promethium-compatible microchip.

The second floor contains a cobalt generator, a curium generator, a ruthenium generator, and a plutonium generator.

The third floor contains a cobalt-compatible microchip, a curium-compatible microchip, a ruthenium-compatible microchip, and a plutonium-compatible microchip.

The fourth floor contains nothing relevant.



In [469]:
F = [
    {'TG','TM'}, # T = promethium
    {'CG','KG','RG','PG'}, # K = Curium
    {'CM','KM','RM','PM'},
    set()
]

from timeit import default_timer as timer

start = timer()
s1 = findMoves(F)
stop = timer()

print("Part 1: objects collected on 4th floor in {} moves".format(s1))
print("Elapsed time = {:.2f} s".format(stop-start))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 
Part 1: objects collected on 4th floor in 33 moves
Elapsed time = 39.98 s


## Part 2

The phase space to explore becomes larger, thus the incresed running time, and BFS is probably not the best algorothm to solve this problem. 

I can see some possible optimization, e.g. when there is more than M-G pairs on a floor, moving one or the other will ultimately bring to the same state, so I could in principle prune those "equivalent stage":

`F1: ...........`  
`F0: AG AM BG MG`

`F1: ..... BG MG` or  `F1: AG AM .....`  
`F0: AG AM .....` or  `F0: ..... BG MG`

`F1: AG AM BG MG`  
`F0: ...........`

I will neverthless leave it running, let's see how long it takes...

In [470]:
extra = {'EG','EM','DG','DM'}
for e in extra:
    F[0].add(e)

from timeit import default_timer as timer

start = timer()
s2 = findMoves(F)
stop = timer()

print("Part 2: objects collected on 4th floor in {} moves".format(s2))
print("Elapsed time = {:d} min".format(int(stop-start)//60))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 
Part 2: objects collected on 4th floor in 57 moves
Elapsed time = 45 min
